<a href="https://colab.research.google.com/github/RajivDalal/hydration-essentials/blob/main/bottle_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Downloading Dataset

In [6]:
!kaggle datasets download -d chethuhn/water-bottle-dataset

Dataset URL: https://www.kaggle.com/datasets/chethuhn/water-bottle-dataset
License(s): CC0-1.0
water-bottle-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [9]:
import zipfile
zip_ref = zipfile.ZipFile('/content/water-bottle-dataset.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

# Image Preprocessing

In [2]:
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
ds = keras.utils.image_dataset_from_directory(
    directory = '/content',
    labels = 'inferred',
    label_mode = 'int',
    batch_size = 32,
    image_size = (256, 256)
)